# Ptolemy's *Geography*




## Notebook configuration

In [ ]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

In [ ]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::ptolemy:1.2.1`

## Retrieve and parse TEI edition

In [ ]:
import scala.xml._

val url = "https://raw.githubusercontent.com/neelsmith/ptolemy/master/tei/tlg0363.tlg009.epist03-p5-u8.xml"
val root = XML.load(url)

In [ ]:
import edu.holycross.shot.ptolemy._

val delimited = TeiParser.parseTEI(root, false)
val ptolStrings = delimited.map(ln => PtolemyString(ln))


## Scale

Use Eratosthenes' estimate of 250,000 stades to scale points from Ptolemy's 180,000 stades.


In [ ]:
case class GeoPoint (lon: Double, lat: Double) {
  def cex(delimiter: String = "#") = {
    Vector(lon, lat).mkString(delimiter)
  }
}
val ratio = 18.0 / 25.0

In [ ]:
val scaled = ptolStrings.map( pt => (GeoPoint(pt.lon * ratio, pt.lat * ratio  ) ))
val scaledGeo = ptolStrings zip scaled

## Adjust baseline for latitude

In [ ]:
val rhodes = 36.0
val offsetRaw = rhodes - (ratio * rhodes)
val offset = BigDecimal(offsetRaw).setScale(2, BigDecimal.RoundingMode.HALF_UP).toDouble


In [ ]:
val shiftedLat = scaledGeo.map { case (pt, geo) => Vector(pt.id,geo.lon,(geo.lat + offset)).mkString(",")}